In [137]:
import pandas as pd
from os import devnull
import xlrd
from datetime import datetime


In [138]:
# Step 1: Read the XLSX template file to get the column names
template_file = 'plantilla.xlsx'
template_df = pd.read_excel(template_file)
template_columns = template_df.columns.tolist()
print(template_columns)

['Date', 'Account', 'Category', 'Subcategory', 'Note', 'Amount', 'Income/Expense', 'Description']


In [139]:
# Step 2: Read the XLS file to get the data you want to parse
data_file = 'Files/05-2024.xls'
# data_df = pd.read_excel(data_file)
wb = xlrd.open_workbook(data_file, logfile=open(devnull, 'w'))
data_df = pd.read_excel(wb, dtype=str,  skiprows=7, engine='xlrd')

In [140]:
# Name the columns of the data_df DataFrame with the first row of the data_df DataFrame
data_df.columns =['fecha', 'concepto', 'fecha valor', 'importe', 'saldo']
print(data_df.head())

        fecha                               concepto fecha valor importe  \
0  01/05/2024              AMAZON* 404-6652105-43     29/04/2024  -69.99   
1  01/05/2024  ENVIO BIZUM pistachos                  01/05/2024   14.35   
2  02/05/2024  PAGO CAJERO 53229502084601450          02/05/2024    -270   
3  02/05/2024  COMPRA T.C. E.S. AGUILAR               02/05/2024   -5.99   
4  03/05/2024  COMPRA T.C. LUPA MAGDALENA GUARDO      03/05/2024  -73.17   

     saldo  
0  2605.97  
1  2620.32  
2  2350.32  
3  2344.33  
4  2271.16  


In [141]:
## Try to convert the first column to a date
try:
	# Convert fecha column with format "%d/%m/%Y" 
	data_df['fecha'] = pd.to_datetime(data_df['fecha'], format='%d/%m/%Y')
 
 # Convert fecha valor column with format "%d/%m/%Y"
	data_df['fecha valor'] = pd.to_datetime(data_df['fecha valor'], format='%d/%m/%Y')
 
	# Print the first 5 rows of the DataFrame
	print(data_df.head())


except ValueError as e:
	# Print the error message
	print(e)
	# Exit the program
	exit()


       fecha                               concepto fecha valor importe  \
0 2024-05-01              AMAZON* 404-6652105-43     2024-04-29  -69.99   
1 2024-05-01  ENVIO BIZUM pistachos                  2024-05-01   14.35   
2 2024-05-02  PAGO CAJERO 53229502084601450          2024-05-02    -270   
3 2024-05-02  COMPRA T.C. E.S. AGUILAR               2024-05-02   -5.99   
4 2024-05-03  COMPRA T.C. LUPA MAGDALENA GUARDO      2024-05-03  -73.17   

     saldo  
0  2605.97  
1  2620.32  
2  2350.32  
3  2344.33  
4  2271.16  


In [142]:
# Try to convert the importe column to a float
try:
    # Convert importe column to float
    data_df['importe'] = data_df['importe'].str.replace(',', '').astype(float)
 
    # Print the first 5 rows of the DataFrame
    print(data_df.head())
except ValueError as e:
    # Print the error message
    print(e)
    # Exit the program
    exit()

       fecha                               concepto fecha valor  importe  \
0 2024-05-01              AMAZON* 404-6652105-43     2024-04-29   -69.99   
1 2024-05-01  ENVIO BIZUM pistachos                  2024-05-01    14.35   
2 2024-05-02  PAGO CAJERO 53229502084601450          2024-05-02  -270.00   
3 2024-05-02  COMPRA T.C. E.S. AGUILAR               2024-05-02    -5.99   
4 2024-05-03  COMPRA T.C. LUPA MAGDALENA GUARDO      2024-05-03   -73.17   

     saldo  
0  2605.97  
1  2620.32  
2  2350.32  
3  2344.33  
4  2271.16  


In [143]:
# Get different pricing data with the same structure ['Amount', 'Note', 'Category', 'Subcategory']
eroski_df = pd.read_excel('Pricing/eroski.xlsx')


# Get general pricing data for general categories ['Procedence', 'Category', 'Subcategory']
general_pricing_df = pd.read_excel('Pricing/general.xlsx')


In [144]:
# Create a dictionary with the different pricing dataframes
procedence_dict = {
  'EROSKI': eroski_df,
}

In [145]:

# Deny list to avoid periodic payments already congifured in app
deny_procedence = ['GITHUB', 'DIGITEAL', 'NOMINA']


In [146]:
# Step 3: Create a new DataFrame to store the parsed data
parsed_data = pd.DataFrame(columns=template_columns)
account = 'K26'

# Step 4: Iterate over the rows of the data_df DataFrame
for index, row in data_df.iterrows():
	# Check the importe column to find matches with the pricing data
	procedence = row['concepto']
	
 	# Check if one of the words in procedence is in the deny_procedence list
	if any(word in procedence for word in deny_procedence):
		continue

	type = 'Expense' if row['importe'] < 0 else 'Income'
 ## date in format %m/%d/%Y
	date = row['fecha'].strftime('%d/%m/%Y')
	amount = abs(row['importe'])
	note = procedence
	category = 'Otros'
	subcategory = None

	selected_pricing_df = None

	# Check the procedence of the row
	for word in procedence_dict.keys():
		if word in procedence:
			selected_pricing_df = procedence_dict[word]
			break
	
 
 
	if selected_pricing_df is not None and type == 'Expense':
		# Check if the amount is in the pricing data
		if amount in selected_pricing_df['Amount'].values:
			# Get the row of the pricing data that matches the amount
			pricing_row = eroski_df[eroski_df['Amount'] == abs(amount)]
			# Get the note, category and subcategory of the pricing data
			note = pricing_row['Note'].values[0] if pricing_row['Note'].any() else None
			category = pricing_row['Category'].values[0] if pricing_row['Category'].any() else None
			subcategory = pricing_row['Subcategory'].values[0] if pricing_row['Subcategory'].any() else None
	else:
		# Check if the procedence is in the general pricing data
		if any(word.lower() in procedence.lower() for word in general_pricing_df['Procedence'].values):
			# Get the row of the general pricing data that matches a word of the procedence
			pricing_row = general_pricing_df[general_pricing_df['Procedence'].str.lower().isin([word.lower() for word in procedence.split()])]
			# Get the category and subcategory of the general pricing data
			category = pricing_row['Category'].values[0] if  pricing_row['Category'].any() else 'Otros'
			subcategory = pricing_row['Subcategory'].values[0] if  pricing_row['Subcategory'].any() else None
			note = pricing_row['Procedence'].values[0] if pricing_row['Procedence'].any() else procedence

   
  	# Insert the parsed row into the parsed_data DataFrame
	new_row = [date, account, category, subcategory, note, amount, type, None]
	parsed_data.loc[len(parsed_data)] = new_row

print(parsed_data.head())
	

 
		
 

         Date Account Category Subcategory  \
0  01/05/2024     K26    Otros        None   
1  01/05/2024     K26    Otros        None   
2  02/05/2024     K26    Otros        None   
3  02/05/2024     K26    Otros        None   
4  03/05/2024     K26    Otros        None   

                                    Note  Amount Income/Expense Description  
0              AMAZON* 404-6652105-43      69.99        Expense        None  
1  ENVIO BIZUM pistachos                   14.35         Income        None  
2  PAGO CAJERO 53229502084601450          270.00        Expense        None  
3  COMPRA T.C. E.S. AGUILAR                 5.99        Expense        None  
4  COMPRA T.C. LUPA MAGDALENA GUARDO       73.17        Expense        None  


In [147]:
# Export as tsv file

# Get the current date
current_date = datetime.now()

# Convert the current date to a string
date_string = current_date.strftime('%d-%m-%Y')

parsed_data.to_csv(f'Results/{date_string}.tsv', sep='\t', index=False)
# Print the date string
print(date_string)


01-06-2024
